# Example of delayedPLIER repository 

Marc Subirana-Granés (2024)

Running the example in the DelayedPLIER repository, I submitted a PR that changes the `setAutoRealizationBackend` function to solve the problem.

# Load libraries/modules

In [3]:
library(TENxPBMCData)
library(here)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats




Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDiffs, rowSds, rowSums2, rowTabulates, rowVarDiffs, rowVars,
    rowWeightedMads, rowWeightedMeans, rowWeigh

# Load data

In [4]:
# define output nb
output_nb_path = here('output/nbs/example_of_delayedPLIER_repository')
dir.create(output_nb_path, showWarnings = FALSE)

path_script_funcs = '/home/msubirana/Documents/pivlab/DelayedPLIER/funcs.R'
priorMat <- readRDS('/home/msubirana/Documents/pivlab/DelayedPLIER/canonicalPathways_ENSG.RDS')

sce <- TENxPBMCData("pbmc4k")
mat <- counts(sce)
rownames(mat) <- rowData(sce)$ENSEMBL_ID 
colnames(mat) <- colData(sce)$Sequence

hdf5_count = file.path(output_nb_path, 'counts.hdf5')
hdf5_dimnames = file.path(output_nb_path, 'dimnames.RDS')

writeHDF5Array(mat, filepath = hdf5_count, name = "count")
saveRDS(list(row.names = rowData(sce)$ENSEMBL_ID , col.names = colData(sce)$Sequence), file = hdf5_dimnames)

see ?TENxPBMCData and browseVignettes('TENxPBMCData') for documentation

loading from cache

Can not create dataset. Object with name 'count' already exists.



ERROR: Error in H5Dcreate(loc$H5Identifier, dataset, tid, sid, dcpl = dcpl): HDF5. Dataset. Unable to initialize object.


In [9]:
head(rowData(sce)$ENSEMBL_ID)

[1] "ENSG00000243485" "ENSG00000237613" "ENSG00000186092" "ENSG00000238009"
[5] "ENSG00000239945" "ENSG00000239906"

[1] "character"

In [7]:
head(colData(sce)$Sequence)

[1] "AAACCTGAGAAGGCCT" "AAACCTGAGACAGACC" "AAACCTGAGATAGTCA" "AAACCTGAGCGCCTCA"
[5] "AAACCTGAGGCATGGT" "AAACCTGCAAGGTTCT"

In [8]:
head(mat)

<6 x 4340> sparse DelayedMatrix object of type "integer":
                AAACCTGAGAAGGCCT ... TTTGTCATCCCAAGAT
ENSG00000243485                0   .                0
ENSG00000237613                0   .                0
ENSG00000186092                0   .                0
ENSG00000238009                0   .                0
ENSG00000239945                0   .                0
ENSG00000239906                0   .                0

In [9]:
source(path_script_funcs) #glmnet and HDF5Array will be loaded 
setAutoRealizationBackend("HDF5Array") #supportedRealizationBackends(), getRealizationBackend()

Loaded glmnet 4.1-8



In [10]:
sce <- DelayedArray(seed = HDF5ArraySeed(filepath = hdf5_count, name = "count"))

In [11]:
dimnamaes <- readRDS(hdf5_dimnames)
rownames(sce) <- dimnamaes$row.names
colnames(sce) <- dimnamaes$col.names

In [12]:
sce[is.na(sce)] <- 0
data <- sce[which(DelayedMatrixStats::rowSds(sce) >0),]

In [13]:
ptm <- proc.time()
output_dplier = file.path(output_nb_path, 'dplier_result')
dir.create(output_dplier, showWarnings = FALSE)
PLIER.res <- PLIER(data, priorMat, output_path = output_dplier)
print(proc.time()-ptm)

Selecting common genes: 5485

Removing 0 pathways with too few genes

Computing SVD

Using rsvd

Done

k is set to 260



[1] 120.8525
[1] "L2 is set to 120.852491861614"
[1] "L1 is set to 60.426245930807"


errorY (SVD based:best possible) = 0.8011

New L3 is 0.000488095243523415

New L3 is 0.000626726698448458

New L3 is 0.000626726698448458

New L3 is 0.000626726698448458

New L3 is 0.000626726698448458

New L3 is 0.000626726698448458

New L3 is 0.000626726698448458

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 151 Bdiff is not decreasing

There are 32  LVs with AUC>0.70



     user    system   elapsed 
15276.188  4761.593  8894.763 


In [14]:
head(data.frame(PLIER.res$B))

,AAACCTGAGAAGGCCT,AAACCTGAGACAGACC,AAACCTGAGATAGTCA,AAACCTGAGCGCCTCA,AAACCTGAGGCATGGT,AAACCTGCAAGGTTCT,AAACCTGCAGGATTGG,AAACCTGCAGGCGATA,AAACCTGCATCCCATC,AAACCTGCATGAAGTA,⋯,TTTGGTTCACTTAAGC,TTTGGTTCAGGACGTA,TTTGGTTCATGCCTAA,TTTGGTTCATTGGCGC,TTTGGTTGTTATCACG,TTTGGTTTCGCTAGCG,TTTGTCACACTTAACG,TTTGTCACAGGTCCAC,TTTGTCAGTTAAGACA,TTTGTCATCCCAAGAT
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"1,REACTOME_RESPIRATORY_ELECTRON_TRANSPORT_ATP_SYNTHESIS_BY_CHEMIOSMOTIC_COUPLING_AND_HEAT_PRODUCTION_BY_UNCOUPLING_PROTEINS_",-0.06041167,-0.020060086,-0.072351252,-0.12441948,-0.188746099,-0.0726506023,-0.106610116,1.07204340,-0.163930761,0.47285977,⋯,-0.1047481930,-0.108844534,-0.06837793,-0.037045499,-0.229603322,0.307604908,-0.04422958,0.145899384,-0.13296022,-0.01949230
"2,PID_IL6_7PATHWAY",0.07883781,0.277803360,0.120966270,-0.10546711,-0.101628431,-0.1262258575,-0.111357255,-0.13368080,-0.100612170,0.70119925,⋯,0.2378427905,-0.139088327,-0.12415270,-0.123831549,-0.054084155,0.488537192,-0.06286200,-0.054330202,-0.09139508,0.21628777
"3,KEGG_PARKINSONS_DISEASE",-0.01690189,-0.009511071,0.015007867,0.01135159,-0.004982491,0.0115287237,0.070999277,-0.16350499,0.006682694,0.01977064,⋯,-0.0002926929,0.114200216,0.02120270,-0.032938852,-0.021550755,-0.035838412,0.00398529,-0.056236183,-0.03048336,-0.03108530
"4,KEGG_PATHOGENIC_ESCHERICHIA_COLI_INFECTION",-0.01210078,-0.012695337,0.006577871,0.02612853,0.006189635,-0.0074906108,-0.039547812,0.04782498,-0.004086115,-0.01897834,⋯,-0.0067904659,-0.008046914,0.05389677,-0.005811486,0.002361694,0.022648748,0.04184630,-0.003403966,0.02205100,-0.01408123
"5,KEGG_NATURAL_KILLER_CELL_MEDIATED_CYTOTOXICITY",-0.03479394,0.006227149,-0.044132464,-0.05190526,-0.035878560,0.0009282842,-0.002472858,-0.14291873,-0.044994070,-0.08778462,⋯,-0.0644967296,-0.020605000,0.11562466,0.150436850,-0.026714169,-0.133844277,0.19659264,0.378261727,-0.04123632,-0.01759173
"6,KEGG_RIBOSOME",-0.24766751,-0.140717597,-0.284247633,-0.13855572,-0.023604823,0.0677065392,-0.302334303,0.47275810,-0.106681150,0.08394256,⋯,-0.2477016588,0.166414938,-0.04910606,-0.102250412,-0.074323910,-0.001567941,-0.12168342,0.371179489,-0.07073071,-0.16114526


In [15]:
head(data.frame(PLIER.res$Z))

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000188976,0.4401267,0.00000000,0.38593902,0,0.22428107,0.3899243,0.1131225,0.032544420,0.008252552,0.09688409,⋯,0.000000000,0.00000000,0.0000000,0.13343190,0.167646177,0.000000000,0.12013283,0.00000000,0.2219445,0.0000000
ENSG00000187608,0.7695049,0.08066534,0.00000000,0,0.00000000,0.1364358,0.1733883,0.000000000,0.293900173,0.00000000,⋯,0.000000000,0.00000000,0.0000000,0.00000000,0.008520085,0.064317960,0.04820271,0.00000000,0.0000000,0.1316259
ENSG00000188157,0.0000000,0.00000000,0.01860792,0,0.01598517,0.0306839,0.3066066,0.000000000,0.026154962,0.00000000,⋯,0.008960603,0.02151207,0.0000000,0.00000000,0.000000000,0.000000000,0.00000000,0.05981510,0.0000000,0.0000000
ENSG00000186891,0.2234135,0.00000000,0.00000000,0,0.55222808,0.1307112,0.0000000,0.000000000,0.000000000,0.00000000,⋯,0.000000000,0.00000000,0.1546808,0.01556843,0.065389959,0.000000000,0.04151722,0.00000000,0.0000000,0.0000000
ENSG00000186827,0.1873396,0.00000000,0.00000000,0,0.02200332,0.2230046,0.0000000,0.009007762,0.000000000,0.00000000,⋯,0.066299523,0.00000000,0.0000000,0.00000000,0.000000000,0.000000000,0.00000000,0.25957265,0.0000000,0.0000000
ENSG00000176022,0.1864918,0.00000000,0.00000000,0,0.00000000,0.3681502,0.0000000,0.044742929,0.000000000,0.00000000,⋯,0.000000000,0.00000000,0.0000000,0.00000000,0.000000000,0.009643393,0.28613783,0.04905919,0.0000000,0.0000000


In [16]:
head(data.frame(PLIER.res$U))

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV251,LV252,LV253,LV254,LV255,LV256,LV257,LV258,LV259,LV260
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PID_CASPASE_PATHWAY,0.08987428,0.0000000,0.000000000,0,0,0.00000000,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
PID_P38ALPHABETADOWNSTREAMPATHWAY,0.00000000,0.0000000,0.000000000,0,0,0.00000000,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
REACTOME_APC_C_CDC20_MEDIATED_DEGRADATION_OF_MITOTIC_PROTEINS,0.00000000,0.0000000,0.000000000,0,0,0.00000000,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
REACTOME_REGULATION_OF_APOPTOSIS,0.00000000,0.0000000,0.000000000,0,0,0.00000000,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
REACTOME_IRON_UPTAKE_AND_TRANSPORT,0.14376437,0.0734511,0.000000000,0,0,0.00000000,0.1246435,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
PID_ERBB1_DOWNSTREAM_PATHWAY,0.01499952,0.0000000,0.002337958,0,0,0.02216941,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
